In [2]:
# Import libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Base path for datasets
data_path = "data"

# Load data
receivals = pd.read_csv(f"{data_path}/kernel/receivals.csv")
purchase_orders = pd.read_csv(f"{data_path}/kernel/purchase_orders.csv")
materials = pd.read_csv(f"{data_path}/extended/materials.csv")
transportation = pd.read_csv(f"{data_path}/extended/transportation.csv")

# Convert date columns to datetime
receivals["date_arrival"] = pd.to_datetime(receivals["date_arrival"], utc=True)
purchase_orders["delivery_date"] = pd.to_datetime(purchase_orders["delivery_date"], utc=True)
purchase_orders["created_date_time"] = pd.to_datetime(purchase_orders["created_date_time"], utc=True)
purchase_orders["modified_date_time"] = pd.to_datetime(purchase_orders["modified_date_time"], utc=True)

### Checking for Negative or Zero Values

We start by checking for negative or zero values in numerical columns where such values are not realistic. These include quantities and weights that should always be strictly positive, such as delivered material weight or vehicle weight.

We focus on the following columns from each dataset:

- **Receivals:** `net_weight`
- **Purchase Orders:** `quantity`
- **Transportation:** `net_weight`, `gross_weight`, `tare_weight`, `vehicle_start_weight` and `vehicle_end_weight`
- **Transportation:**  
  `net_weight`, `gross_weight`, `tare_weight`, `vehicle_start_weight`, `vehicle_end_weight`,  
  `wood`, `ironbands`, `plastic`, `water`, `ice`, `other`, `chips`, `packaging` and `cardboard` 


Identifying these impossible or unexpected values helps us detect potential data entry mistakes, measurement errors, or system inconsistencies that we can investigate further in the cleaning step.


In [6]:
# Function to count zeros and negatives per dataset
def count_dataset_anomalies(df, columns, name):
    for col in columns:
        if col in df.columns:
            num_zero = (df[col] == 0).sum()
            num_negative = (df[col] < 0).sum()
            total = num_zero + num_negative
            print(f"{col}: {total} non-positive values ({num_zero} zero, {num_negative} negative)")

# Columns to check per dataset
receivals_cols = ['net_weight']
purchase_orders_cols = ['quantity']
transportation_cols = [
    'net_weight', 'gross_weight', 'tare_weight', 'vehicle_start_weight', 'vehicle_end_weight',
    'wood', 'ironbands', 'plastic', 'water', 'ice', 'other', 'chips', 'packaging', 'cardboard'
]

# Run anomaly counts
count_dataset_anomalies(receivals, receivals_cols, "Receivals")
count_dataset_anomalies(purchase_orders, purchase_orders_cols, "Purchase Orders")
count_dataset_anomalies(transportation, transportation_cols, "Transportation")


net_weight: 137 non-positive values (137 zero, 0 negative)
quantity: 10 non-positive values (4 zero, 6 negative)
net_weight: 137 non-positive values (137 zero, 0 negative)
gross_weight: 124 non-positive values (124 zero, 0 negative)
tare_weight: 63844 non-positive values (63844 zero, 0 negative)
vehicle_start_weight: 0 non-positive values (0 zero, 0 negative)
vehicle_end_weight: 44 non-positive values (44 zero, 0 negative)
wood: 1110 non-positive values (1110 zero, 0 negative)
ironbands: 6870 non-positive values (6870 zero, 0 negative)
plastic: 6126 non-positive values (6126 zero, 0 negative)
water: 7784 non-positive values (7784 zero, 0 negative)
ice: 7786 non-positive values (7786 zero, 0 negative)
other: 8207 non-positive values (7738 zero, 469 negative)
chips: 7774 non-positive values (7774 zero, 0 negative)
packaging: 6458 non-positive values (6456 zero, 2 negative)
cardboard: 7767 non-positive values (7767 zero, 0 negative)


The anomaly counts from the previous step show that several columns contain zero or negative values. Negative values are physically impossible for these types of measurements and clearly indicate data errors, while zero values may represent either valid cases or missing/unrecorded measurements. 

We will start by examining the negative quantity-values, because they are unequivocal errors. By investigating where these negatives occur, which orders, suppliers, or batches they belong to, and whether they are isolated or systematic, we can better understand the data quality issues and plan an appropriate cleaning strategy.

In [10]:
# Inspect negative values in Purchase Orders
neg_quantity = purchase_orders[purchase_orders['quantity'] < 0]
print(f"Number of negative quantities: {len(neg_quantity)}")
display(neg_quantity)


Number of negative quantities: 6


,purchase_order_id,purchase_order_item_no,quantity,delivery_date,product_id,product_version,created_date_time,modified_date_time,unit_id,unit,status_id,status
0,1,1,-14.0,2003-05-11 22:00:00+00:00,91900143,1,2003-05-12 10:00:48+00:00,2004-06-15 06:16:18+00:00,NaN,NaN,2,Closed
59,205323,10,-380.0,2003-01-30 23:00:00+00:00,1024,1,2003-01-28 13:30:23+00:00,2004-06-15 06:08:40+00:00,40.0,KG,2,Closed
60,205443,10,-390.0,2003-02-09 23:00:00+00:00,1024,1,2003-02-10 09:31:21+00:00,2004-06-15 06:14:46+00:00,40.0,KG,2,Closed
61,206151,10,-260.0,2003-04-20 22:00:00+00:00,1024,1,2003-04-14 09:31:14+00:00,2004-06-15 06:14:50+00:00,40.0,KG,2,Closed
66,206386,10,-340.0,2003-05-13 22:00:00+00:00,1024,1,2003-05-09 08:30:53+00:00,2004-06-15 06:14:54+00:00,40.0,KG,2,Closed
208,209842,10,-8580.0,2004-04-29 22:00:00+00:00,91900170,1,2004-04-28 08:32:31+00:00,2004-06-15 06:16:36+00:00,40.0,KG,2,Closed


We inspected the negative quantity values in the purchase orders. Observing the details of these rows, we notice the following:

- Most negative quantities occur for product_id 1024, but also two other product_ids.
- The magnitude of negative quantities varies significantly, from small values like `-14.0` to extreme cases like `-8580.0`.  
- All affected purchase orders have a `Closed` status, suggesting that they were finalized despite these unusual values.  
- The delivery dates for these orders are all in 2003–2004, indicating this may be historical data issues or corrections.  

Next, we will cross-check these non-positive quantities against the `receivals` table to see if material was actually received.  


2. Decide how to handle these anomalies for analysis: ignore, correct, or flag them.  
3. Document any assumptions or corrections applied to maintain reproducibility and clarity in further analyses.


In [14]:
# Check if any of these have corresponding receivals
matching_receivals = receivals.merge(
    neg_quantity,
    on=['purchase_order_id', 'purchase_order_item_no'],
    how='inner'
)

print("Number of matching rows in receivals:", len(matching_receivals))
if len(matching_receivals) == 0:
    print("No negative quantity purchase orders have matching receivals.")
else:
    print("Some negative quantity purchase orders do have matching receivals:")
    display(matching_receivals.head())

Number of matching rows in receivals: 0
No negative quantity purchase orders have matching receivals.


In [18]:
# Merge purchase orders with receivals to see which have matches
po_with_receivals = purchase_orders.merge(
    receivals,
    on=['purchase_order_id', 'purchase_order_item_no'],
    how='left',
    indicator=True  # This adds a column showing if a match exists
)

# Count how many purchase orders have no corresponding receivals
no_matching_receivals = po_with_receivals[po_with_receivals['_merge'] == 'left_only']
num_no_matches = len(no_matching_receivals)

print(f"Total purchase order items: {len(purchase_orders)}")
print(f"Number of purchase order items without matching receivals: {num_no_matches}")
print(f"Fraction with no matching receivals: {num_no_matches / len(purchase_orders):.2%}")


Total purchase order items: 33171
Number of purchase order items without matching receivals: 10872
Fraction with no matching receivals: 32.78%
